In [1]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 5.6 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
     |████████████████████████████████| 448 kB 65.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from konlpy.tag import Okt
okt = Okt()
token = okt.morphs("나는 자연어 처리를 배운다")
print(token)


['나', '는', '자연어', '처리', '를', '배운다']


In [3]:
#인덱스 부여
word2index = {}
for voca in token :
  if voca not in word2index.keys():
    word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [5]:
#빈도수 순대로 단어를 정렬하는 작업
def one_hot_encoding(word, word2index):
  one_hot_vector = [0]*(len(word2index))
  index = word2index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

one_hot_encoding("자연어",word2index)

[0, 0, 1, 0, 0, 0]

In [6]:
###워드임베딩###
#1;.희소표현
import torch

#원핫벡터 생성
dog = torch.FloatTensor([1,0,0,0,0])
cat = torch.FloatTensor([0,1,0,0,0])
computer = torch.FloatTensor([0,0,1,0,0])
netbook = torch.FloatTensor([0,0,0,1,0])
book = torch.FloatTensor([0,0,0,0,1])

print(torch.cosine_similarity(dog, cat, dim=0))
print(torch.cosine_similarity(cat, computer, dim=0))
print(torch.cosine_similarity(computer, netbook, dim=0))
print(torch.cosine_similarity(netbook, book, dim=0))
#단어의 유사성을 표현하지못함 빠가!!

#2.밀집표현
#사용자가 설정한 값으로 모든 단어의 백터 표현의 차원을 맞춤!(0과 1만 가지는게 아닌 실수값을 가지게 됨)

#3.워드임베딩
# 실수로 학습데이터로 부터 학습!

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


In [1]:
###Wod2Vec 훈련시키기

#1.영어 word2Vec 만들기

import nltk
nltk.download('punkt')

import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

# 전처리
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')

target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

print('총 샘플의 개수 : {}'.format(len(result)))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
총 샘플의 개수 : 273424


In [2]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

# Word2Vec 훈련
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

model_result = model.wv.most_similar("man")
print(model_result)

#Word2Vec 모델저장& 로드
model.wv.save_word2vec_format('./eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

model_result = loaded_model.most_similar("man")
print(model_result)


['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']
[('woman', 0.8621014952659607), ('guy', 0.8228507041931152), ('boy', 0.7803660035133362), ('girl', 0.7797322273254395), ('lady', 0.7794908285140991), ('gentleman', 0.7606490850448608), ('soldier', 0.750495195388794), ('kid', 0.6909241676330566), ('poet', 0.6628771424293518), ('son', 0.6621164083480835)]
[('woman', 0.8621014952659607), ('guy', 0.8228507041931152), ('boy', 0.7803660035133362), ('girl', 0.7797322273254395), ('lady', 0.7794908285140991), ('gentleman', 0.7606490850448608), ('soldier', 0.750495195388794), ('kid', 0.6909241676330566), ('poet', 